# Graph RAG Pattern: Vector + KG using VSS

This supplements notebook 2, where we used LlamaIndex to create a vector store of press releases and linked them to document nodes. In this notebook we demonstrate a graph RAG pattern that performs a hybrid vector/traversal query. It starts by finding similar embeddings. Then it traverses outward to find related events and organizations. 

Here is our data model.

<img src="images/kgc_model.png">

The next figure depicts our design.

<img src="images/kgc_design.png">


In [ ]:
pip install -q llama-index llama-index-vector-stores-neptune llama-index-graph-stores-neptune  llama-index-llms-bedrock llama-index-embeddings-bedrock

### Get the Graph Configuration

In [ ]:
import graph_notebook as gn
config = gn.configuration.get_config.get_config()

region = config.aws_region
graph_identifier=config._host.split(".")[0]

### Imports and Global Settings


In [ ]:
import os
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import StorageContext, VectorStoreIndex, KnowledgeGraphIndex, Settings, load_index_from_storage
from llama_index.core import SimpleDirectoryReader

QA_MODEL="anthropic.claude-3-sonnet-20240229-v1:0"

# define LLM
llm = Bedrock(model=QA_MODEL, 
    model_kwargs={"temperature": 0})
embed_model = BedrockEmbedding(model="amazon.titan-embed-text-v1")

# Set global LLM settings
Settings.llm = llm
Settings.embed_model = embed_model


### Look at the chunks in the vector store

In [ ]:
%%oc

MATCH (n:Chunk) 
CALL neptune.algo.vectors.get(n)
YIELD embedding
RETURN n.file_name as docfile, id(n) as chunkid, n.text as text, embedding
ORDER BY docfile
LIMIT 20


In [ ]:
%%oc

MATCH (d:DOCUMENT)<-[:belongsToDoc]-(c:Chunk)
RETURN id(d) as docid, d.title as title, collect(id(c)) as chunks

## Try Vector Similarity search

In [ ]:
query="Does Amazon have a fulfillment center in Mississippi?"
embedding = embed_model.get_text_embedding(query)
embparams={'emb': embedding}

In [ ]:
%%oc -qp embparams --store-to qres

WITH $emb as emb
CALL neptune.algo.vectors.topKByEmbedding(emb)
YIELD embedding, node, score

// FILTER by good score
WHERE score < 560.0
WITH node, id(node) as chunkid

MATCH  path=(node:Chunk)-[:belongsToDoc]->(d:DOCUMENT)-[ev]->(obs)-[role]->(ent)-[:resolvesToOrg *]->(org)-[:hasKnownPerson|hasParentCompany|hasIndustry *]->(orgrel)
RETURN path
LIMIT 2000

In [ ]:
%%oc -qp embparams --store-to qtext

WITH $emb as emb
CALL neptune.algo.vectors.topKByEmbedding(emb)
YIELD embedding, node, score

// FILTER by good score
WHERE score < 560.0
WITH node, id(node) as chunkid

MATCH  (node:Chunk)-[:belongsToDoc]->(d:DOCUMENT)

RETURN id(node) as chunk_id, node.text as chunk_text, id(d) as doc_id, d.title as doc_title


In [ ]:
%%oc -qp embparams --store-to qres

WITH $emb as emb
CALL neptune.algo.vectors.topKByEmbedding(emb)
YIELD embedding, node, score

// FILTER by good score
WHERE score < 560.0
WITH node, id(node) as chunkid

MATCH  (node:Chunk)-[:belongsToDoc]->(d:DOCUMENT)-[ev]->(obs)-[role]->(ent)-[:resolvesToOrg *]->(org)
OPTIONAL MATCH (org)-[orgev:hasKnownPerson|hasIndustry]->(orgrel)

RETURN distinct id(d) as doc_id, d.title as doc_title, 
    labels(obs) as event, obs.primaryName as event_names,
    type(role) as role_type,
    labels(ent) as entity, collect(distinct ent.names) as entity_names,
    id(org) as org,
    collect(distinct id(orgrel)) as org_relationships
    
LIMIT 2000

In [ ]:
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime')

SYSTEM_PROMPT="You are an expert Q&A system that is trusted around the world.\\nAlways answer the query using the provided context information, and not prior knowledge.\\nSome rules to follow:\\n1. Never directly reference the given context in your answer.\\n2. Avoid statements like \'Based on the context, ...\' or \'The context information ...\' or anything along those lines."

def make_prompt(q, pcontext):
    return "\n".join( [
        "Context information is below.",
        "---------------------",
        pcontext,
        "Given the context information and not prior knowledge, answer the query.",
        f"Query: {q}",
        "Answer: "
    ])

def get_completion(prompt):
    print(prompt)
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[{"role":"user", "content":[{"text": prompt, "type": "text"}]}],
            "temperature": 0.1,
            "system": SYSTEM_PROMPT
        }
    )    
    
    response = bedrock_client.invoke_model(body=body, modelId=QA_MODEL)
    response_body = json.loads(response.get('body').read())
    return response_body.get('content')[0].get('text')
    
get_completion(make_prompt(
    query, 
    str({'textresult': qtext, 'pathresult': qres})
))  